In [1]:
from pytket.circuit import Circuit, fresh_symbol
from pytket.circuit.display import render_circuit_jupyter
from pytket.extensions.quantinuum import QuantinuumBackend
import matplotlib.pyplot as pl

import os
print(os.getcwd())

from pytket.extensions.qiskit import qiskit_to_tk, AerBackend
from pytket.backends.backendresult import BackendResult

import qiskit as qk

import pickle
import pandas as pd
import numpy as np

/Users/utkarshagrawal/Documents/Postdoc/U_1_exp/Quantinum/Weak measurements


In [2]:
which_machine = 'emulator'

In [4]:

if which_machine == 'emulator':
    machine = 'H1-1E'
elif which_machine == 'hardware':
    machine = 'H1-1'

backend = QuantinuumBackend(machine)
# backend.login()

print(machine, "status:", backend.device_state(device_name=machine))

QuantinuumAPIError: HTTP error attempting: Login.

Server Response: {'error': {'code': 1, 'text': 'An error occurred (InvalidParameterException) when calling the InitiateAuth operation: Missing required parameter USERNAME'}}

In [ ]:
file = 'data/compiled_circuits_multiple_ancillas'

# For circuits with scrambling time = 2
# file = 'data/compiled_circuits_multiple_ancillas_t_scram=2'

with open('data/compiled_circuits_multiple_ancillas','rb') as f:
    compiled_circuits = pickle.load(f)

In [ ]:
[param for param in compiled_circuits]

[(14, 14, 0.253, 7),
 (10, 10, 0.306, 4),
 (8, 8, 0.571, 4),
 (14, 14, 0.412, 7),
 (8, 8, 0.783, 4),
 (10, 10, 0.624, 5),
 (10, 10, 0.465, 4),
 (12, 12, 0.942, 5),
 (8, 8, 0.942, 3),
 (10, 10, 0.2, 4),
 (12, 12, 0.2, 5),
 (12, 12, 0.624, 6),
 (10, 10, 0.571, 4),
 (8, 8, 0.306, 4),
 (14, 14, 0.889, 6),
 (14, 14, 0.359, 6),
 (14, 14, 0.677, 6),
 (10, 10, 0.783, 4),
 (14, 14, 0.836, 6),
 (8, 8, 0.465, 4),
 (14, 14, 0.518, 6),
 (10, 10, 0.677, 5),
 (10, 10, 0.518, 5),
 (8, 8, 0.412, 4),
 (10, 10, 0.836, 5),
 (14, 14, 0.783, 7),
 (14, 14, 0.571, 7),
 (12, 12, 0.2, 6),
 (10, 10, 0.359, 5),
 (10, 10, 0.889, 5),
 (8, 8, 0.253, 4),
 (14, 14, 0.2, 6),
 (14, 14, 0.465, 7),
 (14, 14, 0.624, 6),
 (10, 10, 0.412, 4),
 (12, 12, 0.359, 6),
 (12, 12, 0.889, 6),
 (12, 12, 0.677, 6),
 (10, 10, 0.253, 4),
 (14, 14, 0.306, 7),
 (6, 6, 0.942, 3),
 (12, 12, 0.518, 6),
 (12, 12, 0.836, 6),
 (14, 14, 0.889, 7),
 (14, 14, 0.359, 7),
 (8, 8, 0.73, 4),
 (10, 10, 0.571, 5),
 (12, 12, 0.465, 6),
 (14, 14, 0.518, 7)

For $t_scram=5$ we used following values for p 
```python
    p_list = np.round(np.linspace(0.2,0.6*np.pi/2,15),3)[:]
```
 
    

For $t_{scram}=2$ we used following values
```python
    p_list = np.round(np.linspace(0.2*np.pi/2,0.8*np.pi/2,15),3)
```

In [ ]:
n_shots = 200
# p_list = [0.571,0.624,0.677,0.73,0.783][:]
p_list = [0.412,0.518,0.624,0.73,0.836] # this is for t_scram=2
L_list = [8,12,16][:]

def run_emulator():
    count = 0
    handle_dict = {}
    for param,circ in compiled_circuits.items():
        L,depth,p,Q = param
        if L not in L_list:
            continue
        if p not in p_list:
            continue
        if count == 0:
            first_batch = backend.process_circuit(circuit=circ, n_shots=n_shots)
            handle_dict[param] = first_batch
        else:
            new_batch = backend.process_circuit(circuit=circ, n_shots=n_shots)
            handle_dict[param] = new_batch
        count += 1
    return handle_dict


In [ ]:
handle_dict = run_emulator()

In [ ]:
[backend.circuit_status(handle) for handle in handle_dict.values()]

In [ ]:
# Saving and loading handle dictionary

def save_handle_dict(handle_dict,file_dir):
    temp = len(os.listdir(file_dir))
    file_dir = file_dir + str(temp+1) +'/'
    if not os.path.isdir(file_dir):
        os.makedirs(file_dir)
    with open(file_dir + 'handle_dict','wb') as f:
        pickle.dump(handle_dict,f)

    df = pd.DataFrame()
    df['L'] = []
    df['shots'] = []
    df['p'] = []
    display(df)

    number_of_p = len(p_list)
    for param in handle_dict:
        L,depth,p,Q = param
        if Q!= L//2:
            continue
        temp = pd.DataFrame({'L':[L],'p':[p],'shots':[n_shots]})
        df = pd.concat([df,temp],ignore_index=True)

    display(df)
    df.to_csv(file_dir+'information.csv')

def load_handle_dict(file_loc,batch_number):
    file_loc = file_loc +str(batch_number)+'/handle_dict'
    with open(file_loc,'rb') as f:
        handle_dict = pickle.load(f)
    return handle_dict


In [ ]:
if which_machine == 'emulator':
    file_dir = 'data/emulator_data/emulator_handle/'
elif which_machine == 'hardware':
    file_dir = 'data/hardware_data/handle_data/'

# save_handle_dict(handle_dict,file_dir)

In [ ]:
# getting results from the completed jobs
def get_result(handle_dic):
    result_dict = {}
    for param, handle in handle_dic.items():
        result_dict[param] = backend.get_result(handle).to_dict()
    return result_dict

def save_result_dict(result_dict,file_dir,batch_number):
    file_dir = file_dir + str(batch_number) +'/'
    file_loc = file_dir + 'result'
    with open(file_loc,'wb') as f:
        pickle.dump(result_dict,f)

def load_result_dict(file_dir,batch_number):

    if 'result' not in os.listdir(file_dir):
        handle_dict = load_handle_dict(file_dir,batch_number)
        result_dict = get_result(handle_dict)
        save_result_dict(result_dict,file_dir,batch_number)
    else:
        with open(file_dir+ str(batch_number)+'/result','rb') as f:
            result_dict = pickle.load(f)
    return result_dict


In [ ]:
batch_number = 2

result_dict = load_result_dict(file_dir, batch_number)

In [ ]:
print(result_dict.keys())

In [ ]:
## Get outcome data (to be given as input to the decoder code) from the quantinuum emulator/hardware results

### If there is confusion with regards to the outcome data, generate it from scratch for the interested batch numbers. A better data management needs to be thought of.

def outcome_from_results(result,L,depth):
    # bitlist = result.get_bitlist()
    def get_bitlist():
        c_bits = [pytket.Bit('') for _ in range(L*depth)]
        for i in range(L*depth):
            x = i%L
            t = i//L
            temp = ['c'+str(t),[x]]
            c_bits[i] = c_bits[i].from_list(temp)
        return c_bits

    bitlist = get_bitlist()
    measurement_dic = result.get_counts(cbits=bitlist)
    # measurement_dic = {measurements in a single shot:number of occurances}

    outcome_list = []

    for data,N in measurement_dic.items(): # data is measurement outcomes in one shot
        data = np.array(data,dtype=int).reshape((depth,L))
        data = 2*data - 1
        outcome_list.append((data,N))

    return outcome_list


def get_outcomes(file_dir,batch_number,outcome_filedir):
    file_dir = file_dir+str(batch_number)+'/'

    if 'outcomes_generated' in os.listdir(file_dir):
        return ## outcomes were already generated for this batch


    for param, result in result_dict.items():
        print(param)
        result = BackendResult.from_dict(result)
        L,depth,p,Q = param

        ## defining output file name
        if int(depth/L) == 1:
            depth_label = ''
        else:
            depth_label = "_depth_ratio="+str(int(depth/L))
        
        outcome_filedir = outcome_filedir + depth_label + '/'
        
        if not os.path.isdir(outcome_filedir):
            os.makedirs(outcome_filedir)
        
        outcome_file = outcome_filedir + 'L='+str(L)+'_depth='+str(depth)+'_Q='+str(Q)+'_p=' + str(p)+ '_seed='+str(1)
        ########

        if os.path.isfile(outcome_file):
            with open(outcome_file,'rb') as f:
                outcomes = pickle.load(f)
        else:
            outcomes = []

        outcomes = []
        new_outcomes = outcome_from_results(result,L,depth)
        outcomes.extend(new_outcomes)

        with open(outcome_file,'wb') as f:
            pickle.dump(outcomes,f)

    with open(file_dir+'outcomes_generated','wb') as f:
        pickle.dump("",f)


In [ ]:
if which_machine == 'emulator':
    outcome_file_dir = 'data/emulator_data/measurement_data_all_qubits_noisy/'
elif which_machine == 'hardware':
    outcome_file_dir = 'data/hardware_data/measurement_data_all_qubits_noisy/'

get_outcomes(file_dir=file_dir,batch_number=batch_number,outcome_filedir=outcome_file_dir)

## Running the decoder

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import sep_decoder_2 
%reload_ext sep_decoder_2

In [ ]:
if which_machine == 'emulator':
    final_direc = 'data/emulator_data/emulator_sep_data/'
    outcome_direc = 'data/emulator_data/measurement_data_all_qubits_noisy'
L_list = [8,12,16]
# p_list = [0.571,0.624,0.677,0.73,0.783]
p_list = [0.412,0.518,0.624,0.73,0.836]

half_depth=True #when half_depth is true only L//2 layers of outcomes are used for decoding, otherwise L layers are used

sep_decoder_2.collect_sep_data(L_list=L_list,p_list=p_list,depth_ratio=1,half_depth=half_depth
is_noisy=True, final_direc=final_direc,outcomes_filedir=outcome_direc)

8 0.412 4  frac of faulty traj: 0.0  time= 0.14880609512329102
8 0.412 3  frac of faulty traj: 0.0  time= 0.1466202735900879
8 0.518 4  frac of faulty traj: 0.0  time= 0.14717578887939453
8 0.518 3  frac of faulty traj: 0.0  time= 0.14740729331970215
8 0.624 4  frac of faulty traj: 0.01  time= 0.14294815063476562
8 0.624 3  frac of faulty traj: 0.0  time= 0.14519309997558594
8 0.73 4  frac of faulty traj: 0.0  time= 0.13091707229614258
8 0.73 3  frac of faulty traj: 0.0  time= 0.14469408988952637
8 0.836 4  frac of faulty traj: 0.01  time= 0.11221003532409668
8 0.836 3  frac of faulty traj: 0.01  time= 0.14403581619262695
12 0.412 6  frac of faulty traj: 0.0  time= 0.607403039932251
12 0.412 5  frac of faulty traj: 0.0  time= 0.618149995803833
12 0.518 6  frac of faulty traj: 0.0  time= 0.6111071109771729
12 0.518 5  frac of faulty traj: 0.0  time= 0.6133849620819092
12 0.624 6  frac of faulty traj: 0.005  time= 0.5928888320922852
12 0.624 5  frac of faulty traj: 0.0  time= 0.629299879

## Analyzing the data

In [ ]:
def load_data(filename):

    with open(filename,'rb') as f:
        data = pickle.load(f)

    return data

def plot_success_prob(data,L_list,charge_fac = 0,N_samples=-1,marker='o'):
    ent = {}
    err = {}
    for L in L_list[:]:
        ent[L] = []
        err[L] = []
        p_list = list(data[L].keys())
        for p in p_list:
            tempQ = list(data[L][p][L//2])[:N_samples]
            tempQ2 = list(data[L][p][L//2-1])[:N_samples]
            temp = tempQ + tempQ2
            print("L=",L,"p=",p,"data_size:",len(tempQ),len(tempQ2))
           
            ent[L].append(np.average(temp))
            err[L].append(np.std(temp)/(len(temp)-1)**0.5)

        pl.errorbar(np.array(p_list),ent[L],yerr=err[L],ls='-',marker=marker,label='L='+str(L))

    pl.xlabel(r'$\gamma$',fontsize=16)
    # pl.ylabel(r'$\langle p \rangle_{suc}$',fontsize=16)
    pl.ylabel(r'Success prob',fontsize=16)

    pl.legend(fontsize=16)
    pl.tight_layout()

def plot_success_ent(data,L_list,charge_fac = 0,N_samples=-1,marker='o'):
    ent = {}
    err = {}
    for L in L_list[:]:
        ent[L] = []
        err[L] = []
        p_list = list(data[L].keys())
        for p in p_list:
            tempQ = list(data[L][p][L//2])[:N_samples]
            tempQ2 = list(data[L][p][L//2-1])[:N_samples]
            temp = tempQ + tempQ2
            print("L=",L,"p=",p,"data_size:",len(tempQ),len(tempQ2))
            # ent.append(np.sum(np.array(suc_list)>0.5)/len(suc_list))
            ent_list = np.array([(-x*np.log(x) - (1-x)*np.log(1-x)) if 0<x<1 else 0 for x in tempQ2+tempQ])
            ent[L].append(np.average(ent_list))
            err[L].append(np.std(ent_list)/(len(temp)-1)**0.5)

        pl.errorbar(np.array(p_list),ent[L],yerr=err[L],ls='-',marker=marker,label='L='+str(L))

    pl.xlabel(r'$\gamma$',fontsize=16)
    # pl.ylabel(r'$\langle p \rangle_{suc}$',fontsize=16)
    pl.ylabel(r'Success entropy',fontsize=16)

    pl.legend(fontsize=16)
    pl.tight_layout()

def plot_success_binder(data,L_list,N_samples=-1,marker='o',color=None,ls='-',extra_label=''):
    binder = {}
    err = {}
    for L in L_list[:]:
        binder[L] = []
        err[L] = []
        p_list = np.array(list(data[L].keys()))
        for p in p_list:
            tempQ = list(data[L][p][L//2])[:N_samples]
            tempQ2 = list(data[L][p][L//2-1])[:N_samples]
            print("L=",L,"p=",p,"data_size:",len(tempQ),len(tempQ2))
            ent_list = np.array([(-x*np.log(x) - (1-x)*np.log(1-x)) if 0<x<1 else 0 for x in tempQ2+tempQ])
            mu_1 = np.average(ent_list)
            mu_4 = np.average((ent_list-mu_1)**4)
            e_4 = np.std(ent_list**4)/len(tempQ+tempQ2)
            mu_2 = np.average((ent_list-mu_1)**2)
            e_2 = 2*mu_2*np.std(ent_list**2)/len(tempQ+tempQ2)
            binder[L].append(1-mu_4/(3*mu_2**2))
        
        if color is None:
            pl.plot(p_list,binder[L],ls=ls,marker=marker,label='L='+str(L)+extra_label)
        else:
            pl.plot(p_list,binder[L],ls=ls,marker=marker,label='L='+str(L)+extra_label,color=color)

    pl.xlabel(r'$\gamma$',fontsize=16)
    pl.ylabel(r'Binder${}_{suc}$',fontsize=16)
    pl.legend(fontsize=16)
    pl.tight_layout()


def plot_success_variance(data,L_list,N_samples=-1,marker='o',color=None,ls='-',extra_label=''):
    binder = {}
    err = {}
    for L in L_list[:]:
        binder[L] = []
        err[L] = []
        p_list = np.array(list(data[L].keys()))
        for p in p_list:
            tempQ = list(data[L][p][L//2])[:N_samples]
            tempQ2 = list(data[L][p][L//2-1])[:N_samples]
            print("L=",L,"p=",p,"data_size:",len(tempQ),len(tempQ2))

            ent_list = np.array([(-x*np.log(x) - (1-x)*np.log(1-x)) if 0<x<1 else 0 for x in tempQ2+tempQ])

            mu_1 = np.average(ent_list)
            
            mu_2 = np.average((ent_list-mu_1)**2)
            
            binder[L].append(mu_2)
        
        if color is None:
            pl.plot(p_list,binder[L],ls=ls,marker=marker,label='L='+str(L)+extra_label)
        else:
            pl.plot(p_list,binder[L],ls=ls,marker=marker,label='L='+str(L)+extra_label,color=color)

    pl.xlabel(r'$\gamma$',fontsize=16)
    pl.ylabel(r'Binder${}_{suc}$',fontsize=16)
    pl.legend(fontsize=16)
    pl.tight_layout()


In [ ]:
prop_cycle = pl.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

pl.figure(1)


data=load_data('data/emulator_data/emulator_sep_data_constant_depth/seed=1_all_qubits_noisy')
L_list = [8,12,16][:]
plot_success_ent(data,L_list,marker='s',N_samples=-1)
# plot_success_binder(data,L_list,marker='s',N_samples=-1,ls='-', extra_label='_noisy_emulator')



data=load_data('data/emulator_data/emulator_sep_data/seed=1_all_qubits_noisy')
L_list = [8,12,16][:]
plot_success_ent(data,L_list,marker='s',N_samples=-1)
# plot_success_binder(data,L_list,marker='s',N_samples=-1,ls='-', extra_label='_noisy_emulator')
# pl.yscale('log')

-----------------

In [69]:
# file_direc = 'data/hardware_data/handle_data/'
# if not os.path.isdir(file_direc):
#     os.makedirs(file_direc)


# previous_job_number = len(os.listdir(file_direc))
# file_direc = file_direc + str(previous_job_number+1) + '/'
# if not os.path.isdir(file_direc):
#     os.makedirs(file_direc)


# df = pd.DataFrame()
# df['L'] = []
# df['shots'] = []
# df['p'] = []
# display(df)

# number_of_p = len(p_list)
# for L in L_list:
#     for p in p_list:
#         temp = pd.DataFrame({'L':[L],'p':[p],'shots':[n_shots]})
#         df = pd.concat([df,temp],ignore_index=True)

# display(df)
# df.to_csv(file_direc+'information.csv')

# with open(file_direc+'handle_dic','wb') as f:
#     pickle.dump(handle_dict,f)

,L,shots,p


,L,shots,p
0,8.0,200.0,0.571
1,8.0,200.0,0.624
2,8.0,200.0,0.677
3,8.0,200.0,0.730
4,8.0,200.0,0.783
